In [3]:
from pyscf import gto,scf
import pyscf
#from alch_deriv import alch_deriv
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline as spl
import numpy as np
from FcMole import FcM
#from aaff import aaff
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [4]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.05",unit='Bohrs',basis="ccpvqz",verbose=0)
hfCO=scf.RHF(CO)
CO.nao

110

In [5]:
def optimizer2at(a1,a2,fcs=None):
    d1=np.linspace(1.9,2.4,9)
    es=[]
    for d in d1:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpvqz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpvqz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(1.9,2.4,501)
    bspline=spl(d1,es)
    bspline(dspl)
    dminspline=1.9+.001*(bspline(dspl).argmin())
    d2=np.linspace(dminspline-.03,dminspline+.03,7)
    es=[]
    for d in d2:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpvqz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpvqz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(dminspline-.03,dminspline+.03,6001)
    bspline=spl(d2,es)
    #plt.plot(dspl,bspline(dspl))
    COmin=dspl[bspline(dspl).argmin()]
    return COmin,bspline(dspl).min()

In [6]:
optimizer2at("C","O")

(2.0824599999999998, -112.79062626242036)

In [7]:
optimizer2at("N","N")

(2.01366, -108.99447021420647)

In [8]:
optimizer2at("B","F")

(2.3532399999999996, -124.16629187775946)

# GAUGE THE ERROR DUE TO BASIS SET

In [9]:
#CO@NN
optimizer2at("N","N",fcs=[-1,1.])

(2.08115, -112.770598847271)

In [10]:
#CO@BF
optimizer2at("B","F",fcs=[1.,-1.])

(2.08031, -112.75571893712159)

In [11]:
#BF@CO
optimizer2at("C","O",fcs=[-1,1])

(2.35147, -124.14839396550698)

In [12]:
#NN@CO
optimizer2at("C","O",fcs=[1,-1])

(2.0116199999999997, -108.97089704758953)